# Portafolios de Inversión
## Tercer Examen Parcial
José Armando Melchor Soto

---





### Librerías

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch

warnings.filterwarnings('ignore')


---

#### Funciones 

In [ ]:
class OptimizePortfolioWeights:

    def __init__(self, returns: pd.DataFrame, risk_free: float):

        self.rets = returns
        self.cov = returns.cov()
        self.rf = risk_free / 252
        self.n_stocks = len(returns.columns)

    # Min Variance
    def opt_min_var(self):


        var = lambda w: w.T @ self.cov @ w

        w0=np.ones(self.n_stocks)/self.n_stocks

        bounds=[(0, 1)]*self.n_stocks

        constraint=lambda w: sum(w)-1

        result=minimize(fun=var, x0=w0, bounds=bounds,
                        constraints={'fun': constraint, 'type': 'eq'},
                        tol=1e-16)

        return result.x

    # Sharpe Ratio
    def opt_max_sharpe(self):
        rets = self.rets
        rend, cov, rf = self.rets.mean(), self.cov, self.rf

        sr = lambda w: -((np.dot(rend, w) - rf) / ((w.reshape(-1, 1).T @ cov @ w) ** 0.5))

        result = minimize(sr, np.ones(len(rets.T)), bounds=[(0, None)] * len(rets.T),
                          constraints={'fun': lambda w: sum(w) - 1, 'type': 'eq'},
                          tol=1e-16)

        return result.x

    # Semivariance method
    def opt_min_semivar(self, rets_benchmark):

        rets, corr=self.rets.copy(), self.rets.corr()

        diffs=rets-rets_benchmark.values

        below_zero_target=diffs[diffs<0].fillna(0)
        target_downside=np.array(below_zero_target.std())

        target_semivariance=np.multiply(target_downside.reshape(len(target_downside), 1), target_downside) * corr

        semivar = lambda w: w.T @ target_semivariance @ w

        w0=np.ones(self.n_stocks)/self.n_stocks

        bounds=[(0, 1)]*self.n_stocks

        constraint=lambda w: sum(w)-1

        result=minimize(fun=semivar, x0=w0, bounds=bounds,
                        constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)

        return result.x

    # Omega
    def opt_max_omega(self, rets_benchmark):

        rets=self.rets.copy()

        diffs=rets-rets_benchmark.values

        below_zero_target=diffs[diffs<0].fillna(0)
        above_zero_target=diffs[diffs>0].fillna(0)

        target_downside=np.array(below_zero_target.std())
        target_upside=np.array(above_zero_target.std())
        o=target_upside/target_downside

        omega = lambda w: -sum(o * w)


        w0=np.ones(self.n_stocks)/self.n_stocks

        bounds=[(0.05,0.5)]*self.n_stocks

        constraint=lambda w: sum(w)-1

        result=minimize(fun=omega, x0=w0, bounds=bounds,
                        constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)

        return result.x


In [ ]:
class dynamic_backtesting:

    def __init__(self, prices_tactical, prices_strategic, prices_benchmark, capital, rf, months):
        self.prices_tactical = prices_tactical
        self.prices_strategic = prices_strategic
        self.prices_benchmark = prices_benchmark
        self.capital = capital
        self.rf = rf
        self.months = months

    def optimize_weights(self, prices: pd.DataFrame, n_days: int, periods: int):
        start = int(n_days * periods)
        end = int(n_days * (periods + 1))

        temp_data = prices.iloc[start:end, :]
        temp_bench = self.prices_benchmark.iloc[start:end, :]

        temp_rets = temp_data.pct_change().dropna()
        rets_benchmark = temp_bench.pct_change().dropna()

        optimizer = OptimizePortfolioWeights(returns=temp_rets, risk_free=self.rf)

        w_minvar = pd.Series(optimizer.opt_min_var(), index=prices.columns)
        w_sharpe = pd.Series(optimizer.opt_max_sharpe(), index=prices.columns)
        w_semivar = pd.Series(optimizer.opt_min_semivar(rets_benchmark), index=prices.columns)
        w_omega = pd.Series(optimizer.opt_max_omega(rets_benchmark), index=prices.columns)

        return w_minvar, w_sharpe, w_semivar, w_omega

    def simulation(self):
        total_days = len(self.prices_tactical)
        n_periods = round(total_days / 252 * (12 / self.months))
        n_days = round(total_days / n_periods)

        capital = self.capital

        opt_data = self.prices_tactical.iloc[:n_days, :]
        backtesting_tactical = self.prices_tactical.iloc[n_days:, :]
        backtesting_strategic = self.prices_strategic.iloc[n_days:, :]
        backtesting_benchmark = self.prices_benchmark.iloc[n_days:, :]

        rets_tactical = backtesting_tactical.pct_change().dropna()
        rets_strategic = backtesting_strategic.pct_change().dropna()
        rets_benchmark = backtesting_benchmark.pct_change().dropna()

        min_len = min(len(rets_tactical), len(rets_strategic), len(rets_benchmark))
        rets_tactical = rets_tactical.iloc[:min_len, :]
        rets_strategic = rets_strategic.iloc[:min_len, :]
        rets_benchmark = rets_benchmark.iloc[:min_len, :]

        minvar, sharpe, semivar, omega = [capital], [capital], [capital], [capital]
        day_counter, periods_counter = 0, 0

        w_minvar, w_sharpe, w_semivar, w_omega = self.optimize_weights(opt_data, n_days, 0)

        # Pesos estratégicos fijos uniformes
        w_strategic = pd.Series(
            [1 / self.prices_strategic.shape[1]] * self.prices_strategic.shape[1],
            index=self.prices_strategic.columns
        )

        for day in range(min_len - 1):
            if day_counter == n_days:
                w_minvar, w_sharpe, w_semivar, w_omega = self.optimize_weights(backtesting_tactical, n_days, periods_counter)
                periods_counter += 1
                day_counter = 0

            combined_minvar = w_minvar.add(w_strategic, fill_value=0)
            combined_sharpe = w_sharpe.add(w_strategic, fill_value=0)
            combined_semivar = w_semivar.add(w_strategic, fill_value=0)
            combined_omega = w_omega.add(w_strategic, fill_value=0)

            combined_minvar /= combined_minvar.sum()
            combined_sharpe /= combined_sharpe.sum()
            combined_semivar /= combined_semivar.sum()
            combined_omega /= combined_omega.sum()

            rets_combined = pd.concat([
                rets_tactical.iloc[day, :],
                rets_strategic.iloc[day, :]
            ])

            minvar.append(minvar[-1] * (1 + (rets_combined @ combined_minvar)))
            sharpe.append(sharpe[-1] * (1 + (rets_combined @ combined_sharpe)))
            semivar.append(semivar[-1] * (1 + (rets_combined @ combined_semivar)))
            omega.append(omega[-1] * (1 + (rets_combined @ combined_omega)))

            day_counter += 1

        # Capital acumulado benchmark
        capital_benchmark = capital * (1 + rets_benchmark.sum(axis=1)).cumprod()
        capital_benchmark = capital_benchmark.iloc[:len(minvar)-1]  # igual largo

        df = pd.DataFrame({
            'Date': backtesting_tactical.index[:len(minvar)-1],
            'Min Var': minvar[:-1],
            'Sharpe': sharpe[:-1],
            'Semivar': semivar[:-1],
            'Omega': omega[:-1],
            'Benchmark': capital_benchmark.values
        }).set_index('Date')

        return df


---